In [6]:
import pandas as pd
import networkx as nx

In [5]:
data = pd.read_csv("data/edgelist.csv", index_col=0)
data

,Person 1,Person 2
1,Admin.mgr1,Admin.2
2,Sales.mgr3,Sales.8
3,Production.2,Production.3
4,Production.2,Production.7
5,Sales.1,Sales.2
...,...,...
136,Production.mgr1,Production.2
137,Sales.Dir,Sales.6
138,Admin.4,Production.11
139,Admin.mgr1,Admin.3


In [ ]:
G = nx.from_pandas_edgelist(df, source="")